In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import tensorflow as tf
from tensorflow import keras

print("GPU available:", len(tf.config.list_physical_devices('GPU')))

2023-05-20 04:11:55.594384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 04:11:56.388312: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU available: 1


2023-05-20 04:11:57.517854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-20 04:11:57.555635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-20 04:11:57.558492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
from data_preprocessing import DataProcessor, Dataset
from loss import WeightedMSE
from var_autoencoder import MLPVarAutoencoder
from model import OCCModel

In [4]:
PROJECT_PATH = '/raid/shumnov/music-style-performer/'

In [5]:
BATCH_SIZE = 64

In [6]:
absolute_velocities = [True, False]
weights = [(1, 2), (1, 1)]
latent_dim = [40, 20]
architecture = [((256, 128), (0.2, 0.2), (128, 256), (0.2, 0.2)),
                ((64, 32), (0, 0), (32, 64), (0, 0))]

In [7]:
def test(av, ld, w, arch, name):
    dp = DataProcessor(
        notes_qty=20, 
        include_first_tone=False, 
        absolute_velocities=av
    )
    
    dset = Dataset(dp, BATCH_SIZE)
    dset.load_dset(PROJECT_PATH + 'data/dset20.npy', PROJECT_PATH + 'data/test_dset20.npy')
    
    #dset.train = dset.train[:1000]
    #dset.test = dset.test[:1000]
    #dset.vel_mess = dset.vel_mess[:1000]
    #dset.leg_mess = dset.leg_mess[:1000]
    
    loss = WeightedMSE(
        vel_mask=dp.vel_mask, 
        leg_mask=dp.leg_mask, 
        first_last_mask=dp.first_last_mask, 
        vel_weight=w[0], 
        leg_weight=w[1], 
        first_last_weight=1
    )
    
    autoencoder = MLPVarAutoencoder(
        input_size=dp.input_size,
        latent_dim=ld,
        noise=0.1,
        encoder_layers=arch[0],
        encoder_dropout=arch[1],
        decoder_layers=arch[2],
        decoder_dropout=arch[3],
        mse_loss=loss
    )
    
    model = OCCModel(
        autoencoder=autoencoder,
        dataprocessor=dp,
        dist_weight=1, 
        vel_weight=1, 
        leg_weight=2
    )
    model.compile(
        modelsdir=PROJECT_PATH+'traindata/vae',
        name=name, 
        optimizer=keras.optimizers.Adam(3e-4),
        loss=loss,
        ckpt_epochs=1
    )
    model.summary()
    
    model.fit(dset, 10, 10)

In [ ]:
for av in range(len(absolute_velocities)):
    for w in range(len(weights)):
        for ld in range(len(latent_dim)):
            for a in range(len(architecture)):
                test(absolute_velocities[av], latent_dim[ld], weights[w], architecture[a], f'{a}{w}{ld}{av}')

2023-05-20 04:12:19.316963: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-20 04:12:19.320196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-20 04:12:19.321827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2023-05-20 04:12:26.769157: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x21715810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-20 04:12:26.769204: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2023-05-20 04:12:26.775558: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-20 04:12:27.011079: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-20 04:12:27.087699: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2023-05-20 04:12:27.133958: I ./tensorflow/compiler/jit/device_compiler.h:180] Com